Agrotech Live | Wiggle Labs<br>
Jeremy Barton <br>
06.16.2025

# Session Data Exploratory Analysis

Read in the session data from the export directory.

In [1]:
import pandas as pd
sesh_df = pd.read_csv('/home/nunya/Documents/GitHub/AGT/files/export/2025-06-10.csv')
sesh_df.drop_duplicates(subset='MAC', keep='first')

,Timestamp,MAC,Temperature,Moisture,Light,Conductivity
0,2025-05-21 20:08:39,c4:7c:8d:6d:24:9e,23.2,3,86,0
1,2025-05-21 20:08:43,c4:7c:8d:6d:28:fa,9.3,42,177,2055
2,2025-05-21 20:08:47,c4:7c:8d:6d:4e:df,9.6,50,131,2355
3,2025-05-21 20:08:52,c4:7c:8d:6d:26:c9,9.6,27,150,1600


Add a column to the dataframe called "Subject Name", indicating the name of the thing we are analyzing.

In [2]:
sub_nme = {'Subject Name'}
sesh_df['Subject'] = ' '

Replace all occurances of the entire MAC address with just the last five characters capturing only the unique identifiers.

In [3]:
sesh_df['MAC'] = sesh_df['MAC'].str[-5:]
sesh_df

,Timestamp,MAC,Temperature,Moisture,Light,Conductivity,Subject
0,2025-05-21 20:08:39,24:9e,23.2,3,86,0,
1,2025-05-21 20:08:43,28:fa,9.3,42,177,2055,
2,2025-05-21 20:08:47,4e:df,9.6,50,131,2355,
3,2025-05-21 20:08:52,26:c9,9.6,27,150,1600,
4,2025-05-21 20:19:16,24:9e,23.2,3,89,0,
...,...,...,...,...,...,...,...
3493,2025-06-10 16:15:02,26:c9,25.4,43,5232,537,
3494,2025-06-10 16:25:26,24:9e,24.9,6,69,0,
3495,2025-06-10 16:25:29,28:fa,25.1,38,3956,1317,
3496,2025-06-10 16:25:33,4e:df,25.1,54,3385,1557,


Assign names to the different MAC addresses based on their unique suffixes.

`.loc()` allows us to access rows and columns directly using labels or a boolean array. `str()` converts the mac to a string to use `.endswith()` and search for the correct names to assign.

In [4]:
sesh_df.loc[sesh_df['MAC'].str.endswith('26:c9'), 'Subject'] = 'Romaine'
sesh_df.loc[sesh_df['MAC'].str.endswith('4e:df'), 'Subject'] = 'Simpson'
sesh_df.loc[sesh_df['MAC'].str.endswith('28:fa'), 'Subject'] = 'Kale'
sesh_df.loc[sesh_df['MAC'].str.endswith('24:9e'), 'Subject'] = 'Buttercrunch'

sesh_df.head(3)

,Timestamp,MAC,Temperature,Moisture,Light,Conductivity,Subject
0,2025-05-21 20:08:39,24:9e,23.2,3,86,0,Buttercrunch
1,2025-05-21 20:08:43,28:fa,9.3,42,177,2055,Kale
2,2025-05-21 20:08:47,4e:df,9.6,50,131,2355,Simpson


Move the Subject field up front.

In [5]:
sesh_df.insert(0, 'Subject', sesh_df.pop('Subject'))
sesh_df.head(4)

,Subject,Timestamp,MAC,Temperature,Moisture,Light,Conductivity
0,Buttercrunch,2025-05-21 20:08:39,24:9e,23.2,3,86,0
1,Kale,2025-05-21 20:08:43,28:fa,9.3,42,177,2055
2,Simpson,2025-05-21 20:08:47,4e:df,9.6,50,131,2355
3,Romaine,2025-05-21 20:08:52,26:c9,9.6,27,150,1600


Find the seven rows in the session with the lowest value for Temperature.

In [6]:
lowest_temp = sesh_df.nsmallest(7, 'Temperature')
lowest_temp

,Subject,Timestamp,MAC,Temperature,Moisture,Light,Conductivity
69,Kale,2025-05-22 00:03:11,28:fa,8.4,41,191,1932
77,Kale,2025-05-22 00:34:54,28:fa,8.4,41,160,1932
79,Kale,2025-05-22 00:45:35,28:fa,8.4,41,191,1918
47,Kale,2025-05-21 22:48:20,28:fa,8.5,41,207,1951
50,Kale,2025-05-21 22:58:59,28:fa,8.5,41,224,1954
53,Kale,2025-05-21 23:09:41,28:fa,8.5,41,69,1954
56,Kale,2025-05-21 23:20:17,28:fa,8.5,41,146,1954


This is raw data, and we'll need to make sense of it eventually. Use melt() to make it completely long format and include everything.

In [7]:
sd_melt = sesh_df.melt(
    id_vars=['Timestamp','MAC','Subject'],
    var_name='Feature',
    value_name='Value'
)

sd_melt

,Timestamp,MAC,Subject,Feature,Value
0,2025-05-21 20:08:39,24:9e,Buttercrunch,Temperature,23.2
1,2025-05-21 20:08:43,28:fa,Kale,Temperature,9.3
2,2025-05-21 20:08:47,4e:df,Simpson,Temperature,9.6
3,2025-05-21 20:08:52,26:c9,Romaine,Temperature,9.6
4,2025-05-21 20:19:16,24:9e,Buttercrunch,Temperature,23.2
...,...,...,...,...,...
13987,2025-06-10 16:15:02,26:c9,Romaine,Conductivity,537.0
13988,2025-06-10 16:25:26,24:9e,Buttercrunch,Conductivity,0.0
13989,2025-06-10 16:25:29,28:fa,Kale,Conductivity,1317.0
13990,2025-06-10 16:25:33,4e:df,Simpson,Conductivity,1557.0


Now pivot the melted dataframe so that the index contains the dates.

In [8]:
pivot_df = sd_melt.pivot_table(
    index='Timestamp',
    columns='Feature',
    values='Value'
)
pivot_df

Feature,Conductivity,Light,Moisture,Temperature
Timestamp,,,,
2025-05-21 20:08:39,0.0,86.0,3.0,23.2
2025-05-21 20:08:43,2055.0,177.0,42.0,9.3
2025-05-21 20:08:47,2355.0,131.0,50.0,9.6
2025-05-21 20:08:52,1600.0,150.0,27.0,9.6
2025-05-21 20:19:16,0.0,89.0,3.0,23.2
...,...,...,...,...
2025-06-10 16:15:02,537.0,5232.0,43.0,25.4
2025-06-10 16:25:26,0.0,69.0,6.0,24.9
2025-06-10 16:25:29,1317.0,3956.0,38.0,25.1


<small>© Wiggle Labs 2025<small>